In [1]:
## Задание 3.3 Команды spark

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

import pyspark.sql.types as T
import pyspark.sql.functions as F

from datetime import datetime

conf = SparkConf()
conf.setMaster("local").setAppName('My app')

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


print('Запущен Spark версии', spark.version)

Запущен Spark версии 3.3.1


**Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты**
       

·   id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

·   timestamp – дата и время события в формате unix timestamp.

·   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

·   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

·   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.

·   sign – наличие у пользователя личного кабинета. Значения – True/False.

In [2]:
esquema = T.StructType([
    T.StructField("id", T.IntegerType(),False),
    T.StructField("timestamp", T.LongType(),True),
    T.StructField("type", T.StringType(),True),
    T.StructField("page_id", T.IntegerType(),True),
    T.StructField("tag", T.StringType(),True),
    T.StructField("sing", T.BooleanType(),True)
])


**b. Создайте датафрейм с описанной выше схемой данных.**

**c. Наполните датафрейм данными. Пример:**

    (12345, 1667627426, "click", 101, "Sport”, False)

In [3]:
data     =[(1, 1637627426, "visit", 101, 'Sport', True),
           (1, 1637621667, "scroll", 101, 'Sport', True),
           (1, 1637621620, "click", 101, 'Sport', True),
           (1, 1637621762, "visit", 102, 'Politics', True),
           (1, 1637628565, "click", 102, 'Politics', True),
           (1, 1637625861, "visit", 103, 'Sport', True),
           (2, 1637628001, "visit", 104, 'Politics', True),
           (2, 1637628201, "scroll", 104, 'Politics', True),
           (2, 1637628151, "click", 104, 'Politics', True),
           (2, 1638628200, "visit", 105, 'Business', True),
           (2, 1638628226, "click", 105, 'Business', True),
           (2, 1637628317, "visit", 106, 'Business', True),
           (2, 1637628359, "scroll", 106, 'Business', True),
           (3, 1637628422, "visit", 101, 'Sport', False),
           (3, 1637828486, "scroll", 101, 'Sport', False),
           (4, 1637628505, "visit", 106, 'Business', False),
           (5, 1637628511, "visit", 101, 'Sport', True),
           (5, 1637628901, "click", 101, 'Sport', True),
           (4, 1637628926, "visit", 102, 'Politics', False),
           (5, 1637628976, "click", 102, 'Politics', True),
           (4, 1637999111, "click", 101, 'Politics', False),
           (4, 1638888999, "click", 102, 'Politics', False)
          ]

df = spark.createDataFrame(data=data,schema=esquema)

In [4]:
df = df.select(*[i for i in df.columns if i != "timestamp"],
                 F.from_unixtime("timestamp").alias("event_time"))

In [5]:
df.show(50)

+---+------+-------+--------+-----+-------------------+
| id|  type|page_id|     tag| sing|         event_time|
+---+------+-------+--------+-----+-------------------+
|  1| visit|    101|   Sport| true|2021-11-23 07:30:26|
|  1|scroll|    101|   Sport| true|2021-11-23 05:54:27|
|  1| click|    101|   Sport| true|2021-11-23 05:53:40|
|  1| visit|    102|Politics| true|2021-11-23 05:56:02|
|  1| click|    102|Politics| true|2021-11-23 07:49:25|
|  1| visit|    103|   Sport| true|2021-11-23 07:04:21|
|  2| visit|    104|Politics| true|2021-11-23 07:40:01|
|  2|scroll|    104|Politics| true|2021-11-23 07:43:21|
|  2| click|    104|Politics| true|2021-11-23 07:42:31|
|  2| visit|    105|Business| true|2021-12-04 21:30:00|
|  2| click|    105|Business| true|2021-12-04 21:30:26|
|  2| visit|    106|Business| true|2021-11-23 07:45:17|
|  2|scroll|    106|Business| true|2021-11-23 07:45:59|
|  3| visit|    101|   Sport|false|2021-11-23 07:47:02|
|  3|scroll|    101|   Sport|false|2021-11-25 15

**Вывести топ-5 самых активных посетителей сайта**

In [6]:
df.groupby("id").count().orderBy("count",ascending=False).show(5)

+---+-----+
| id|count|
+---+-----+
|  2|    7|
|  1|    6|
|  4|    4|
|  5|    3|
|  3|    2|
+---+-----+



**Посчитать процент посетителей, у которых есть ЛК**

In [7]:
persent = 100/(df.count()/df.filter(df.sing==True).count())

In [8]:
print(persent,'%')

72.72727272727273 %


**Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице**

In [9]:
df_click = df.filter(df.type=="click")
df_click.groupBy("page_id").count().orderBy("count",ascending=False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    101|    3|
|    102|    3|
|    105|    1|
|    104|    1|
+-------+-----+



**Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)**

In [10]:
dfTime = df.withColumn("new",F.floor(F.hour("event_time") / F.lit(4)))
dfTime.show(50)

+---+------+-------+--------+-----+-------------------+---+
| id|  type|page_id|     tag| sing|         event_time|new|
+---+------+-------+--------+-----+-------------------+---+
|  1| visit|    101|   Sport| true|2021-11-23 07:30:26|  1|
|  1|scroll|    101|   Sport| true|2021-11-23 05:54:27|  1|
|  1| click|    101|   Sport| true|2021-11-23 05:53:40|  1|
|  1| visit|    102|Politics| true|2021-11-23 05:56:02|  1|
|  1| click|    102|Politics| true|2021-11-23 07:49:25|  1|
|  1| visit|    103|   Sport| true|2021-11-23 07:04:21|  1|
|  2| visit|    104|Politics| true|2021-11-23 07:40:01|  1|
|  2|scroll|    104|Politics| true|2021-11-23 07:43:21|  1|
|  2| click|    104|Politics| true|2021-11-23 07:42:31|  1|
|  2| visit|    105|Business| true|2021-12-04 21:30:00|  5|
|  2| click|    105|Business| true|2021-12-04 21:30:26|  5|
|  2| visit|    106|Business| true|2021-11-23 07:45:17|  1|
|  2|scroll|    106|Business| true|2021-11-23 07:45:59|  1|
|  3| visit|    101|   Sport|false|2021-

**Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.** 

In [11]:
# cгруппируем результаты таблицы предыдущего задания по значениям столбца new
df2 = dfTime.groupby("new")\
    .agg(F.count("*").alias("event_count"))\
    .orderBy("event_count",ascending=False)


In [12]:
df2.registerTempTable("df_table")
spark.sql("SELECT new FROM df_table WHERE event_count = (SELECT MAX(event_count) FROM df_table)").show()

/home/peter/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+---+
|new|
+---+
|  1|
+---+



Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

    1. Id – уникальный идентификатор личного кабинета
    2. User_id – уникальный идентификатор посетителя
    3. ФИО посетителя
    4. Дату рождения посетителя 
    5. Дата создания ЛК

 **Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.**

In [13]:
#cтруктура таблицы личного кабинета
esquemaAreaPersonal = T.StructType([
    T.StructField("id", T.IntegerType(),True),
    T.StructField("user_id", T.IntegerType(),True),
    T.StructField("fio", T.StringType(),True),
    T.StructField("dbd", T.DateType(),True),
    T.StructField("dpa", T.DateType(),True),
])

from datetime import datetime

# заполним данными
data_ap = [
    (101,1,"Никифоров Август Арестович",datetime.strptime("2000-05-05", "%Y-%m-%d"),datetime.strptime("2021-10-29", "%Y-%m-%d")),
    (102,2,"Кудимов Алексей Алексеевич",datetime.strptime("1998-07-13", "%Y-%m-%d"),datetime.strptime("2021-11-22", "%Y-%m-%d")),
    (105,5,"Музин Владимир Владимирович",datetime.strptime("1995-02-10", "%Y-%m-%d"),datetime.strptime("2021-11-23", "%Y-%m-%d"))
]

df_ap = spark.createDataFrame(data = data_ap, schema = esquemaAreaPersonal)

In [14]:
# таблица по личным кабинетам
df_ap.show()

+---+-------+--------------------+----------+----------+
| id|user_id|                 fio|       dbd|       dpa|
+---+-------+--------------------+----------+----------+
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|
|102|      2|Кудимов Алексей А...|1998-07-13|2021-11-22|
|105|      5|Музин Владимир Вл...|1995-02-10|2021-11-23|
+---+-------+--------------------+----------+----------+



In [15]:
#объединенная таблица
df_all = df_ap.alias("lk").join(dfTime.alias("web"),
                               on = [F.col("lk.user_id")==F.col("web.id")],
                               how = "left")

In [16]:
df_all.show(100)

+---+-------+--------------------+----------+----------+---+------+-------+--------+----+-------------------+---+
| id|user_id|                 fio|       dbd|       dpa| id|  type|page_id|     tag|sing|         event_time|new|
+---+-------+--------------------+----------+----------+---+------+-------+--------+----+-------------------+---+
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1| visit|    103|   Sport|true|2021-11-23 07:04:21|  1|
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1| click|    102|Politics|true|2021-11-23 07:49:25|  1|
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1| visit|    102|Politics|true|2021-11-23 05:56:02|  1|
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1| click|    101|   Sport|true|2021-11-23 05:53:40|  1|
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1|scroll|    101|   Sport|true|2021-11-23 05:54:27|  1|
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29|  1| visit|    101|   Sport|true

In [17]:
#Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
df_all.registerTempTable("df_table")
spark.sql("SELECT DISTINCT fio FROM df_table WHERE tag = 'Sport' AND type = 'visit'").show()


+--------------------+
|                 fio|
+--------------------+
|Никифоров Август ...|
|Музин Владимир Вл...|
+--------------------+



**Выведите 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.** 

In [18]:
df1 = df_all.select(
      F.col("user_id"),  
      F.col("dpa"),
      F.col("event_time"),  
      F.datediff(F.col("event_time"),F.col("dpa")).alias("datediff"))


df1.groupby("user_id").max("datediff").show()

+-------+-------------+
|user_id|max(datediff)|
+-------+-------------+
|      1|           25|
|      5|            0|
|      2|           12|
+-------+-------------+



In [19]:
# пользовательская функция определения пола
def getGender(fio):
    surname, name, middlename = fio.split(" ")
    if (((surname[-2:] == "ов") or (surname[-2:] == "ев")) and (middlename[-3:] == "вич")):
        return "m"
    else:
        return "w"